In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import collections
import pandas as pd
import seaborn as sns
import matplotlib as mp
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import LogNorm

sns.set()

In [111]:
# Read simulation files
def datafileread(measurename,skipfirstrows):
    # Reading Datafiles
    path = measurename
    data = np.genfromtxt(path,
                        skip_header=skipfirstrows,
                        delimiter=',',
                        dtype=(float,float),
                        unpack=True)
    return data

def loadModel(filename, skiprows, offset=0):
    # measurement
    tlp_v,tlp_length,nominal,amplitude,width = datafileread(filename,skiprows)

    tlp_length = np.round(tlp_length * 1e9, decimals=1)
    width *= 1e9

    # Add DC voltage value
    # It's a simplification over the actual input voltage
    # but we need consistent values for the x-axis otherwise the x-values cannot be matched from one simulation to another 
    tlp_v += offset
    
    print("## Model: %s" % filename)
    print("Width : %E (ns) -> %E (ns)" % (np.amin(width), np.amax(width)))
    print("Amplitude : %E (V) -> %E (V)" % (np.amin(tlp_v), np.amax(tlp_v)))
    
    df_amplitude = pd.DataFrame({'x': tlp_length, 'y': tlp_v, 'z': amplitude})
    df_amplitude = df_amplitude.pivot(index='y',columns='x', values='z')

    df_width = pd.DataFrame({'x': tlp_length, 'y': tlp_v, 'z': width})
    df_width = df_width.pivot(index='y',columns='x', values='z')
    
    return df_amplitude, df_width

def findClosest(dataframe, xin, yin):
    xnearest = dataframe.columns.map(lambda x: abs(xin-x)).argsort()
    ynearest = dataframe.index.map(lambda y: abs(yin-y)).argsort()
    return xnearest[0], ynearest[0]

def findNextPair(df_x, df_y, xin, yin):
    #
    x,y = findClosest(df_x, xin, yin)
    #print(df_x.columns)
    #print("x[%f vs %f] (ns) & y[%f vs %f] (V)" % (xin, df_x.columns[x],yin, df_x.index[y]))
    print("d %f vs %f (ns)" % (xin, df_x.columns[x]))
    print("d %f vs %f (ns)" % (yin, df_x.index[y]))
    
    if x <= 0 or x >= len(df_x.columns) - 1:
        print("Warning : x index (%d) at boundary" % x)
        
    if y <= 0 or y >= len(df_x.index) - 1:
        print("Warning : y index (%d) at boundary" % y)
       
        
    outputX = df_x.iloc[y,x]
    
    #
    x,y = findClosest(df_y, xin, yin)
    #print(df_x.index)
    #print("x[%f vs %f] (ns) & y[%f vs %f] (V)" % (xin, df_y.columns[x],yin, df_y.index[y]))
    outputY = df_y.iloc[y,x]
   
        
    if x <= 0 or x >= len(df_y.columns) - 1:
        print("Warning : x index (%d) at boundary" % x)
        #print("%f vs %f (ns)" % (xin, df_y.columns[x]))
       
    
    if y <= 0 or y >= len(df_y.index) - 1:
        print("Warning : y index (%d) at boundary" % y)
        #print("%f vs %f (ns)" % (yin, df_y.index[y]))
        
    
    return outputX, outputY

vpre_df_amp, vpre_df_width = loadModel('cz_vpre_V2.csv', 19, offset=0)
bg_df_amp, bg_df_width = loadModel('cz_bandgap_V2.csv', 14, offset=9)
reg_df_amp, reg_df_width = loadModel('cz_regulator_V2.csv', 14, offset=1)

## Model: cz_vpre_V2.csv
Width : 7.761300E-01 (ns) -> 1.548790E+03 (ns)
Amplitude : -5.000000E+01 (V) -> -2.000000E+00 (V)
## Model: cz_bandgap_V2.csv
Width : 2.640160E-02 (ns) -> 9.957310E+03 (ns)
Amplitude : -6.000000E+00 (V) -> 8.000000E+00 (V)
## Model: cz_regulator_V2.csv
Width : -1.455830E+04 (ns) -> 5.676870E+03 (ns)
Amplitude : -1.000000E+00 (V) -> 9.000000E-01 (V)


In [114]:
# Input TLP stress characteristics

inputX = 1000 # ns
inputY = -50 # V

# Apply the models

pair1 = findNextPair(vpre_df_width, vpre_df_amp, inputX, inputY)
print("Vpre done")
pair2 = findNextPair(bg_df_width, bg_df_amp, pair1[0], pair1[1])
print("Bandgap done")
pair3 = findNextPair(reg_df_width, reg_df_amp, pair2[0], pair2[1])

print("##                       Vpre output values : (%.0f ns, %.2f V)" % pair1)
print("##                    Bandgap output values : (%.0f ns, %.2f V)" % pair2)
print("##                  Regulator output values : (%.0f ns, %.2f V)" % pair3)

d 1000.000000 vs 1000.000000 (ns)
d -50.000000 vs -50.000000 (ns)
Vpre done
d 1000.000000 vs 923.700000 (ns)
d -5.331840 vs -5.000000 (ns)
Bandgap done
d 877.280000 vs 923.700000 (ns)
d -1.653780 vs -1.000000 (ns)
##                       Vpre output values : (1000 ns, -5.33 V)
##                    Bandgap output values : (877 ns, -1.65 V)
##                  Regulator output values : (291 ns, 2.27 V)
